In [1]:
import netCDF4
import xarray as xr
import matplotlib.pyplot as plt
params = {'figure.figsize' : (18, 10),'legend.fontsize': 16,'xtick.labelsize':16,'ytick.labelsize':16,'axes.labelsize':16,'font.size':16}
plt.rcParams.update(params)

import sys
sys.path.insert(0,"/home/ad/alberta/git/xscale")
import xscale

import xscale.spectral.fft as xfft
from xscale.spectral.tools import plot_spectrum
import xscale.signal.generator as xgen

from dask_jobqueue import PBSCluster
from dask.distributed import Client
import dask.dataframe as dd
import dask

import zarr
import os
import glob
import numpy as np

%matplotlib inline


In [2]:
cluster = PBSCluster(
    cores=1,
    memory="120GB",
    project='pangeo',
    processes=1,
    walltime='04:00:00',
    local_directory='$TMPDIR')
cluster

/home/ad/alberta/git/conda-pack/pytide/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44009 instead
  http_address["port"], self.http_server.port


PBSCluster('tcp://10.120.43.24:46854', workers=0, threads=0, memory=0 B)

In [3]:
cluster.scale(20)

In [4]:
client = Client(cluster) # scheduler_file="/home/ad/briolf/scheduler.json")
client

Client Scheduler: tcp://10.120.43.24:46854 Dashboard: http://10.120.43.24:44009/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
%%time
root = "/work/ALT/odatis/eNATL60/BLBT02/gridT-2D/"
files = sorted(glob.glob(root+'/*.nc'))

drop_vars = [
    'nav_lat',
    'nav_lon',
    'somxl010',
    'sosaline',
    'sosstsst']


open_kwargs = dict(drop_variables=(drop_vars),decode_cf=True, concat_dim="time_counter") #, combine='nested')
dsall = xr.open_mfdataset(files, combine='nested',parallel=True, **open_kwargs)
dsall

CPU times: user 7.12 s, sys: 869 ms, total: 7.99 s
Wall time: 5min 36s


,Array,Chunk
Bytes,93.50 kB,192 B
Shape,"(11688,)","(24,)"
Count,1461 Tasks,487 Chunks
Type,datetime64[ns],numpy.ndarray
,Array,Chunk
Bytes,187.01 kB,384 B
Shape,"(11688, 2)","(24, 2)"
Count,1461 Tasks,487 Chunks
Type,datetime64[ns],numpy.ndarray
,Array,Chunk


In [6]:
dsg=xr.open_dataset('/work/ALT/odatis/eNATL60/mesh_mask_eNATL60_3.6_lev1.nc4')

In [7]:
box=(-90,-60,30,45)
domaineNATL60=(box[0]<dsg.nav_lon)*(dsg.nav_lon<box[1])*(box[2]<dsg.nav_lat)*(dsg.nav_lat<box[3])
whereeNATL60=np.where(domaineNATL60)
jmineNATL60 = whereeNATL60[0].min()
jmaxeNATL60 = whereeNATL60[0].max()
imineNATL60 = whereeNATL60[1].min()
imaxeNATL60 = whereeNATL60[1].max()

print(jmineNATL60,jmaxeNATL60+1,imineNATL60,imaxeNATL60+1)
START_DATE = np.datetime64('2009-07-01')
END_DATE = np.datetime64('2010-06-30')
time_series=dsall['time_counter']
period = (time_series >= START_DATE) & (time_series <= END_DATE)
time=time_series[period]
sshall=dsall.sossheig[period,jmineNATL60:jmaxeNATL60+1,imineNATL60:imaxeNATL60+1]
t=time.values

1510 2800 474 2299


In [ ]:
%%time

dsfilt=xr.open_mfdataset("/work/ALT/odatis/eNATL60/outputs/pytide/eNATL60GS-BLBT02_tide-direct_time_M2_y2009m07-y2010m06_k*.nc",combine='nested',
                     concat_dim='time_counter',parallel=True,decode_cf=True) 
dsfilt

In [ ]:
%%time

ssh_all_GS=sshall[0:8735,2000-jmineNATL60+1,2000-imineNATL60+1]
ssh_pytide_GS=dsfilt.tide[0:8735,2000-jmineNATL60+1,2000-imineNATL60+1]
ssh_filt_GS=ssh_all_GS-ssh_pytide_GS
time=dsall.time_counter[0:8735]

In [ ]:
%%time

ssh_all_GS.load()
ssh_pytide_GS.load()
ssh_filt_GS.load()
time.load()

In [ ]:
ssh_filt_GS.shape


In [ ]:
fig=plt.figure(figsize=(20,5))
plt.plot(time[0:8710],ssh_all_GS[0:8710],'k',label='eNATL60 in GS total')
plt.plot(time[0:8710],ssh_pytide_GS[0:8710],'g',label='eNATL60 in GS pytide')
plt.plot(time[0:8710],ssh_filt_GS[0:8710],'r',label='eNATL60 in GS filt')
plt.legend()

In [ ]:
fig=plt.figure(figsize=(20,5))
plt.plot(time[0:1000],ssh_all_GS[0:1000],'k',label='eNATL60 in GS total')
plt.plot(time[0:1000],ssh_pytide_GS[0:1000],'g',label='eNATL60 in GS pytide')
plt.plot(time[0:1000],ssh_filt_GS[0:1000],'r',label='eNATL60 in GS filt')
plt.legend()

In [ ]:
fig=plt.figure(figsize=(20,5))
plt.plot(time[0:100],ssh_all_GS[0:100],'k',label='eNATL60 in GS total')
plt.plot(time[0:100],ssh_pytide_GS[0:100],'g',label='eNATL60 in GS pytide')
plt.plot(time[0:100],ssh_filt_GS[0:100],'r',label='eNATL60 in GS filt')
plt.legend()
plt.savefig('time_series_ssh_filt_pytide_GS_M2.png')

In [ ]:
NPADDING = 2
SPtime_h = xfft.fft(ssh_all_GS,dim='time_counter', dx=1., detrend='mean',nfft=ssh_all_GS.shape[0]*NPADDING,tapering=True)
MEANPSD_h=xfft.psd(SPtime_h).load()
freqs_h=MEANPSD_h.f_time_counter.values
freqs_h = freqs_h[NPADDING::] 


In [ ]:
NPADDING = 2
SPtime_pytide = xfft.fft(ssh_pytide_GS,dim='time_counter', dx=1., detrend='mean',nfft=ssh_pytide_GS.shape[0]*NPADDING,tapering=True)
MEANPSD_pytide=xfft.psd(SPtime_pytide).load()
freqs_pytide=MEANPSD_pytide.f_time_counter.values
freqs_pytide = freqs_pytide[NPADDING::] 


In [ ]:
NPADDING = 2
SPtime_hp = xfft.fft(ssh_filt_GS,dim='time_counter', dx=1., detrend='mean',nfft=ssh_filt_GS.shape[0]*NPADDING,tapering=True)
MEANPSD_hp=xfft.psd(SPtime_hp).load()
freqs_hp=MEANPSD_hp.f_time_counter.values
freqs_hp = freqs_hp[NPADDING::] 


In [ ]:
import matplotlib.pyplot as plt
params = {'figure.figsize' : (18, 10),'legend.fontsize': 16,'xtick.labelsize':16,'ytick.labelsize':16,'axes.labelsize':16,'font.size':16}
plt.rcParams.update(params)
fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)

ax.loglog(freqs_h,MEANPSD_h[NPADDING::],'k',label='total')
ax.loglog(freqs_hp,MEANPSD_hp[NPADDING::],'r',label='filt')
#ax.loglog(freqs_pytide,MEANPSD_pytide[NPADDING::],'g',label='pytide')
#ax.set_xlim(1e-3,1)
ax.grid(which='both',axis='both')
plt.xlabel('Frequency (cph)')
plt.ylabel('PSD (m2/cph)')
plt.legend()
plt.title('Temporal Spectrum of eNATL60 SSH in GS')

In [ ]:
import matplotlib.pyplot as plt
params = {'figure.figsize' : (18, 10),'legend.fontsize': 16,'xtick.labelsize':16,'ytick.labelsize':16,'axes.labelsize':16,'font.size':16}
plt.rcParams.update(params)
fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)

#ax.loglog(freqs_h,MEANPSD_h[NPADDING::],'k',label='total')
#ax.loglog(freqs_hp,MEANPSD_hp[NPADDING::],'r',label='filt')
ax.loglog(freqs_pytide,MEANPSD_pytide[NPADDING::],'g',label='pytide')
#ax.set_xlim(1e-3,1)
ax.grid(which='both',axis='both')
plt.xlabel('Frequency (cph)')
plt.ylabel('PSD (m2/cph)')
plt.legend()
plt.title('Temporal Spectrum of eNATL60 SSH in GS')

In [ ]:
import matplotlib.pyplot as plt
params = {'figure.figsize' : (18, 10),'legend.fontsize': 16,'xtick.labelsize':16,'ytick.labelsize':16,'axes.labelsize':16,'font.size':16}
plt.rcParams.update(params)
fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)

ax.loglog(freqs_h,MEANPSD_h[NPADDING::],'k',label='total')
ax.loglog(freqs_hp,MEANPSD_hp[NPADDING::],'r',label='filt')
ax.loglog(freqs_pytide,MEANPSD_pytide[NPADDING::],'g',label='pytide')
#ax.set_xlim(1e-3,1)
ax.grid(which='both',axis='both')
plt.xlabel('Frequency (cph)')
plt.ylabel('PSD (m2/cph)')
plt.legend()
plt.title('Temporal Spectrum of eNATL60 SSH in GS')

In [ ]:
import matplotlib.pyplot as plt
params = {'figure.figsize' : (18, 10),'legend.fontsize': 16,'xtick.labelsize':16,'ytick.labelsize':16,'axes.labelsize':16,'font.size':16}
plt.rcParams.update(params)
fig = plt.figure(facecolor='white')
ax = fig.add_subplot(111)

ax.loglog(freqs_h,MEANPSD_h[NPADDING::],'k',label='total')
ax.loglog(freqs_hp,MEANPSD_hp[NPADDING::],'r',label='filt')
ax.loglog(freqs_pytide,MEANPSD_pytide[NPADDING::],'g',label='pytide')
ax.set_xlim(1e-2,1e-1)
ax.grid(which='both',axis='both')
plt.xlabel('Frequency (cph)')
plt.ylabel('PSD (m2/cph)')
plt.legend()
plt.title('Temporal Spectrum of eNATL60 SSH in GS')